# 💰 Subscription Fatigue Predictor
## AI-Driven Economic Intelligence for Churn Prediction & Pricing Optimization

---

**Project Track**: Predictive Analytics / Time-Series Forecasting / Causal Inference

**Author**: Subscription Fatigue Predictor Team  
**Version**: 2.5.2  
**Last Updated**: January 2026

---

## 1. Problem Definition & Objective

### 🎯 Problem Statement

In today's saturated digital subscription economy, **subscription fatigue** has emerged as a critical challenge. Consumers are increasingly overwhelmed by the number of recurring payments, making them highly sensitive to price increases. A poorly timed or excessive price hike can trigger catastrophic churn events.

### 📌 Key Challenges

| Challenge | Business Impact |
|-----------|----------------|
| **Price Sensitivity Detection** | When does a price increase cross the tolerance threshold? |
| **Competitive Resonance** | How do competitors' pricing decisions affect our retention? |
| **Early Warning Signals** | Can we detect churn intent before it manifests? |
| **Segment Heterogeneity** | How do different customer segments respond to price changes? |

### 🌍 Real-World Motivation

- **Netflix** lost 200K subscribers in 2022 after announcing a $2/month price increase
- **Disney+** saw accelerated growth when pricing remained competitive with HBO Max
- Search volume for "Cancel Netflix" spikes 3-4 weeks after price announcements

### 🎓 Learning Objectives

This notebook demonstrates:
1. **Price elasticity modeling** using econometric techniques
2. **Churn risk prediction** with XGBoost gradient boosting
3. **Heterogeneous treatment effects** using Causal Forest (EconML)
4. **Change point detection** for identifying structural breaks in time series
5. **Game-theoretic pricing** with Bertrand competition models

## 2. Environment Setup & Dependencies

First, let's configure our environment and import all necessary modules.

In [ ]:
# Standard Library
import os
import sys
import warnings
from pathlib import Path

# Set project root for imports
PROJECT_ROOT = Path(os.getcwd()).parent
sys.path.insert(0, str(PROJECT_ROOT))

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Data Science Stack
import numpy as np
import pandas as pd
import sqlite3
from datetime import datetime, timedelta

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    roc_auc_score, classification_report, confusion_matrix
)

# Statistical Analysis
import ruptures as rpt
from scipy import stats

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Environment configured successfully!")
print(f"📁 Project Root: {PROJECT_ROOT}")

In [ ]:
# Import Project Modules
from src.data.collectors.data_ingestion import DataIngestionPipeline
from src.models.ml.ml_models import ChurnRiskPredictor, HeterogeneousEffectAnalyzer
from src.models.economic.economic_models import ElasticityCalculator, BertrandCompetitionModel
from src.models.statistical.statistical_models import ChangePointDetector, CausalAnalyzer
from src.models.advanced_models import (
    BundleOptimizer, PsychographicSegmenter, 
    CompetitiveResonanceModel, WeeklyChurnDetector
)

print("✅ Project modules imported successfully!")

## 3. Data Understanding & Preparation

### 📊 Dataset Overview

| Table | Description | Key Fields |
|-------|-------------|------------|
| `companies` | Streaming service providers | name, sector, stock_symbol |
| `pricing_history` | Historical price evolution | effective_date, price, change_percentage |
| `subscriber_metrics` | Subscriber counts & KPIs | subscriber_count, churn_rate, arpu |
| `search_trends` | Cancellation search intent | search_term, search_volume |
| `content_calendar` | Upcoming content releases | title, tier, quality_score |
| `real_world_churn_data` | **Real** Kaggle Telco Dataset | gender, SeniorCitizen, Tenure, Churn |

### 🔄 Data Source

This notebook uses a hybrid approach:
1. **Synthetic Data**: Generated via `DataIngestionPipeline` for time-series pricing dynamics.
2. **Real-World Data**: Ingested automatically from **Kaggle** (Telco Customer Churn) using your private API credentials.

In [ ]:
# Initialize Database & Fetch Kaggle Data
DB_PATH = PROJECT_ROOT / 'data' / 'subscription_fatigue.db'

print("🔧 Running ingestion pipeline (including Kaggle check)...")
pipeline = DataIngestionPipeline(str(DB_PATH))
pipeline.run_full_pipeline()
print("✅ Pipeline execution complete!")

In [ ]:
# Load Data from SQLite
def load_data(db_path):
    """Load all tables from the SQLite database."""
    conn = sqlite3.connect(str(db_path))
    
    tables = {}
    try:
        tables['companies'] = pd.read_sql("SELECT * FROM companies", conn)
        tables['pricing'] = pd.read_sql("SELECT * FROM pricing_history", conn)
        tables['metrics'] = pd.read_sql("SELECT * FROM subscriber_metrics", conn)
        tables['trends'] = pd.read_sql("SELECT * FROM search_trends", conn)
        tables['content'] = pd.read_sql("SELECT * FROM content_calendar", conn)
    except Exception as e:
        print(f"⚠️ Some tables may be missing: {e}")
    finally:
        conn.close()
    
    return tables

data = load_data(DB_PATH)

# Display table summaries
for table_name, df in data.items():
    print(f"\n📦 {table_name}: {len(df):,} rows, {len(df.columns)} columns")
    print(f"   Columns: {', '.join(df.columns[:5])}{'...' if len(df.columns) > 5 else ''}")

### 🌍 Real-World Data Analysis (Kaggle)

Here we validate our models against real customer churn data from the Kaggle dataset fetched during ingestion.

In [ ]:
# Load Real Kaggle Data
conn = sqlite3.connect(str(DB_PATH))
try:
    real_df = pd.read_sql("SELECT * FROM real_world_churn_data", conn)
    print(f"✅ Successfully loaded real-world dataset: {len(real_df):,} rows")
    display(real_df.head())
    
    if 'Churn' in real_df.columns:
        real_churn_rate = (real_df['Churn'] == 'Yes').mean() * 100
        synth_churn_rate = data['metrics']['churn_rate'].mean()
        
        print(f"\n📊 Dataset Comparison:")
        print(f"   Real-World Avg Churn:  {real_churn_rate:.2f}%")
        print(f"   Synthetic Avg Churn:   {synth_churn_rate:.2f}%")
except Exception as e:
    print("ℹ️ Real-world data table not found. Kaggle API may not have been configured correctly.")
finally:
    conn.close()

### 📊 Exploratory Data Analysis (EDA)

In [ ]:
# EDA 1: Pricing Timeline Across Services
fig = px.line(
    data['pricing'].merge(data['companies'], on='company_id'),
    x='effective_date',
    y='price',
    color='name',
    title='💲 Subscription Pricing Evolution (2020-2025)',
    labels={'price': 'Monthly Price (USD)', 'effective_date': 'Date', 'name': 'Service'},
    template='plotly_dark'
)
fig.update_layout(legend=dict(orientation='h', yanchor='bottom', y=1.02))
fig.show()

## 4. Model / System Design

### 🏗️ Architecture Overview

The Subscription Fatigue Predictor employs a **multi-model ensemble** approach:

```
┌──────────────────────────────────────────────────────────────────┐
│                    PREDICTION PIPELINE                          │
├──────────────────────────────────────────────────────────────────┤
│                                                                  │
│   ┌─────────────────┐    ┌─────────────────┐    ┌────────────┐  │
│   │ Data Ingestion  │───▶│  Feature Store  │───▶│  ML Models │  │
│   │   Pipeline      │    │ (Synth + Real)  │    │            │  │
│   └─────────────────┘    └─────────────────┘    └────────────┘  │
│                                                        │        │
│                                                        ▼        │
│   ┌─────────────────┐    ┌─────────────────┐    ┌────────────┐  │
│   │  Economic       │◀───│   Aggregation   │◀───│  Causal    │  │
│   │  Equilibrium    │    │     Layer       │    │  Forest    │  │
│   └─────────────────┘    └─────────────────┘    └────────────┘  │
│                                                                  │
└──────────────────────────────────────────────────────────────────┘
```

## 5. Core Implementation

### 🤖 Model 1: XGBoost Churn Predictor

In [ ]:
# [Rest of original model implementation code...]

## 6. Evaluation & Analysis

### 📏 Performance Metrics

In [ ]:
# [Rest of original evaluation code...]

## 7. Ethical Considerations

### ⚖️ Responsible AI Framework

## 8. Conclusion & Future Scope